# Deep Learning Notes

## Embedding vectors

See paper [here](http://dl.acm.org/citation.cfm?id=2999959)

For protein sequences, I have seen that using hashing for k-mers works well, however, it would be interesting to use something similar to embeddings, a way for encoding words depending on frequency. 

I am not sure if keras can do this, but tensorflow provides a good example. In any case this is just an input encoding, basically independent of the learning process, It could be done using tensorflow if keras is not helpful on this. The basic principle is to maximize the average log probability within some context:

$$ \frac{1}{T} \sum_{t=1}^{T} \sum_{-c \leq j \leq c, j \neq 0} \log p( \omega_{t+j} | \omega_t) $$

## Reaction encoding

It looks like fingerprint difference between products and substrates works fine. Ideally, we could do an embedding using ECFP diameters, but I think that it is not yet implemented in RDKit. Otherwise, Morgan embedding should work for initial testing. I will work here on developing some routines that encode reactions for a neural network at multiple diameters.

In [6]:
import numpy as np
def getReactants(equation):
    reactants = {}
    for x in equation.split(' + '):
        n, c = x.split(' ')
        try:
            n = int(n)
        except:
            pass
        reactants[c] = n
    return reactants

def reacSubsProds(dbfile):
    rsp = {}
    for l in open(dbfile):
        if l.startswith('#'):
            continue
        m = l.rstrip().split('\t')
        rid = m[0]
        subs = {}
        prods = {}
        m = l.rstrip().split('\t')
        left, right = m[1].split(' = ')
        subs = getReactants(left)
        prods = getReactants(right)
        ec = m[4]
        if len(subs) > 0 and len(prods) > 0:
            rsp[rid] = (subs, prods, ec)
    return rsp


In [7]:
rsp = reacSubsProds('/Users/mibsspc2/Documents/data/METANETX2/reac_prop.tsv')

In [10]:
fp = np.load('/Users/mibsspc2/Documents/data/fp/mgfp5.npz')

In [22]:
import rdkit
import rdkit.DataStructs
rdkit.DataStructs.ConvertToExplicit(fp['x'][0])

ArgumentError: Python argument types in
    rdkit.DataStructs.cDataStructs.ConvertToExplicit(UIntSparseIntVect)
did not match C++ signature:
    ConvertToExplicit(SparseBitVect const*)

In [26]:
fp['x'][0].GetNonzeroElements()

{195323148: 2,
 1016841875: 2,
 1239451710: 1,
 2246699815: 1,
 2246997334: 1,
 3696389118: 1}

In [30]:
fp['x'][0].ToBinary()

b"\x01\x00\x00\x00\x04\x00\x00\x00\xff\xff\xff\xff\x06\x00\x00\x00\x0ce\xa4\x0b\x02\x00\x00\x00\x93\xc6\x9b<\x02\x00\x00\x00>\x88\xe0I\x01\x00\x00\x00'\xeb\xe9\x85\x01\x00\x00\x00Vu\xee\x85\x01\x00\x00\x00\xfekR\xdc\x01\x00\x00\x00"

In [38]:
# We can make arithmetic operations for the reaction fingerprint
fp2 = fp['x'][1] - fp['x'][0]

In [41]:
rdkit.DataStructs.FoldToTargetDensity(fp2)

AttributeError: 'UIntSparseIntVect' object has no attribute 'GetNumOnBits'

In [37]:
dir(rdkit.DataStructs)

['AllBitSimilarity',
 'AllProbeBitsMatch',
 'AsymmetricSimilarity',
 'BitVectToBinaryText',
 'BitVectToFPSText',
 'BitVectToText',
 'BraunBlanquetSimilarity',
 'BulkAllBitSimilarity',
 'BulkAsymmetricSimilarity',
 'BulkBraunBlanquetSimilarity',
 'BulkCosineSimilarity',
 'BulkDiceSimilarity',
 'BulkKulczynskiSimilarity',
 'BulkMcConnaugheySimilarity',
 'BulkOnBitSimilarity',
 'BulkRogotGoldbergSimilarity',
 'BulkRusselSimilarity',
 'BulkSokalSimilarity',
 'BulkTanimotoSimilarity',
 'BulkTverskySimilarity',
 'ComputeL1Norm',
 'ConvertToExplicit',
 'ConvertToNumpyArray',
 'CosineSimilarity',
 'CreateFromBinaryText',
 'CreateFromBitString',
 'CreateFromFPSText',
 'DiceSimilarity',
 'DiscreteValueType',
 'DiscreteValueVect',
 'EIGHTBITVALUE',
 'ExplicitBitVect',
 'FOURBITVALUE',
 'FPBReader',
 'FingerprintSimilarity',
 'FoldFingerprint',
 'FoldToTargetDensity',
 'InitFromDaylightString',
 'IntSparseIntVect',
 'KulczynskiSimilarity',
 'LongSparseIntVect',
 'McConnaugheySimilarity',
 'MultiFP

## Comment
Instead of using Morgan fingerprints, it seems to be better using RDKIT fingerprints (diameter can be controlled). It generates binary fingerprints that can be folded.


In [45]:
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.inchi import MolFromInchi

In [47]:
pinocembrin = 'InChI=1S/C15H12O4/c16-10-6-11(17)15-12(18)8-13(19-14(15)7-10)9-4-2-1-3-5-9/h1-7,13,16-17H,8H2'
pino = MolFromInchi(pinocembrin)
pinof1 = RDKFingerprint(pino, minPath=1, maxPath=1)
pinof2 = RDKFingerprint(pino, minPath=1, maxPath=2)
pinof3 = RDKFingerprint(pino, minPath=1, maxPath=3)
pinof4 = RDKFingerprint(pino, minPath=1, maxPath=4)
pinof5 = RDKFingerprint(pino, minPath=1, maxPath=5)


In [50]:
pinof5.ToBitString()

'000000000000000000000000000000000000000000001000000000000000000000000010000101000000000000011100000000010000000000100000000000000101010000000000000010100000000001000000000001001000000000000000001010000000010011000001000000100000000001000000001000100000000000100000000000000000010000100101000000100000000000000000010000000000001010100001100000001010000000000000010000001100000000000000000100000000010000000001000001000000000000000000000001000101000000000000010001001000000100000000001100000000000010010010000010000100000000000100000000000000000000000000000000100011000000000001000001000001100000001000000001001100000000000100100000001000000000000000101000000000010000000000000000000000000000001000000000001000010010000000000100000001000100001000100000000000000010000000001000000010010000000000101000000000000000000000000000001010000000000000001000000100001100000000100000001100000000000000000000100001000000000000000100000101000000100000000000000000000100100001000000000000000001100000010000000100000

In [53]:
pinof1.ToBitString()

'000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000

In [16]:
from rdkit.Chem.rdmolfiles import MolFromSmiles
from rdkit.Chem.rdmolops import RDKFingerprint
import csv
import sys
csv.field_size_limit(sys.maxsize)
def reacfp(radius=5):
    smiles = {}
    fps = {}
    rfp = {}
    with open('../../data/METANETX2/reac_smi.csv') as f:
        for row in csv.DictReader(f):
            rid = row['RID']
            smi = row['SMILES']
            left, right = smi.split('>>')
            rleft = left.split('.')
            rright = right.split('.')
            ok = True
            mleft = []
            mright = []
            for c in rleft:
                if c not in smiles:
                    try:
                        smiles[c] = MolFromSmiles(c)
                    except:
                        ok = False
                        break
                mleft.append((c, smiles[c]))
            if not ok:
                continue
            for c in rright:
                if c not in smiles:
                    try:
                        smiles[c] = MolFromSmiles(c)
                    except:
                        ok = False
                        break
                mright.append((c, smiles[c]))
            if not ok:
                continue
            for c in mright:
                if c[0] not in fps:
                    try:
                        fps[c[0]] = RDKFingerprint(c[1], minPath=1, maxPath=radius)
                    except:
                        ok = False
                        break
                if rid not in rfp:
                    rfp[rid] = fps[c[0]]
                else:
                    rfp[rid] = rfp[rid] | fps[c[0]]
            if not ok:
                del rfp[rid]
            for c in mleft:
                if c[0] not in fps:
                    try:
                        fps[c[0]] = RDKFingerprint(c[1], minPath=1, maxPath=radius)
                    except:
                        ok = False
                        break
                if rid not in rfp:
                    rfp[rid] = fps[c[0]]
                else:
                    rfp[rid] = rfp[rid] ^ fps[c[0]]
            if not ok:
                del rfp[rid]
                continue
            print(rid)
    return rfp
            
                        
            

In [17]:
rfp = reacfp()

MNXR01
MNXR1
MNXR10
MNXR1000
MNXR10005
MNXR10007
MNXR1001
MNXR10012
MNXR10015
MNXR1002
MNXR10020
MNXR10030
MNXR10033
MNXR10051
MNXR10055
MNXR10061
MNXR10062
MNXR1007
MNXR101
MNXR10113
MNXR10128
MNXR1013
MNXR10135
MNXR1017
MNXR10192
MNXR102
MNXR10200
MNXR10201
MNXR1022
MNXR10221
MNXR10236
MNXR10267
MNXR10274
MNXR1028
MNXR10299
MNXR103
MNXR10302
MNXR10303
MNXR10312
MNXR10316
MNXR10322
MNXR10332
MNXR10335
MNXR10341
MNXR1035
MNXR10354
MNXR10355
MNXR1036
MNXR1037
MNXR1038
MNXR1039
MNXR104
MNXR1040
MNXR1041
MNXR10411
MNXR10419
MNXR1042
MNXR10420
MNXR10421
MNXR10422
MNXR1043
MNXR10436
MNXR1045
MNXR10451
MNXR1047
MNXR1048
MNXR1049
MNXR105
MNXR1050
MNXR10505
MNXR1051
MNXR1052
MNXR1055
MNXR1056
MNXR1057
MNXR10574
MNXR10598
MNXR106
MNXR1061
MNXR1064
MNXR1067
MNXR1068
MNXR1069
MNXR10696
MNXR10704
MNXR10705
MNXR1071
MNXR10716
MNXR10737
MNXR10756
MNXR10759
MNXR10761
MNXR10768
MNXR10769
MNXR10775
MNXR10776
MNXR10784
MNXR10792
MNXR10795
MNXR10797
MNXR10798
MNXR108
MNXR1085
MNXR10876
MNXR109
MNXR1097
M

MNXR16738
MNXR16739
MNXR16741
MNXR16742
MNXR16749
MNXR16754
MNXR16758
MNXR16760
MNXR16761
MNXR16762
MNXR16765
MNXR16771
MNXR16777
MNXR16782
MNXR16783
MNXR16784
MNXR16785
MNXR16786
MNXR16793
MNXR16798
MNXR1680
MNXR16803
MNXR16804
MNXR16805
MNXR16808
MNXR16810
MNXR16845
MNXR16848
MNXR16849
MNXR16857
MNXR16860
MNXR16861
MNXR16862
MNXR16864
MNXR16865
MNXR16867
MNXR16868
MNXR16869
MNXR16870
MNXR16871
MNXR16872
MNXR16876
MNXR16880
MNXR16881
MNXR16896
MNXR16897
MNXR16899
MNXR169
MNXR1690
MNXR1691
MNXR1692
MNXR16920
MNXR16927
MNXR1693
MNXR16930
MNXR16933
MNXR16936
MNXR16937
MNXR16938
MNXR16939
MNXR16942
MNXR16943
MNXR16944
MNXR16951
MNXR16953
MNXR16954
MNXR16959
MNXR16960
MNXR16961
MNXR16962
MNXR16963
MNXR16964
MNXR16965
MNXR16966
MNXR16967
MNXR16969
MNXR16971
MNXR16972
MNXR16973
MNXR16986
MNXR16987
MNXR16993
MNXR16994
MNXR16995
MNXR17
MNXR170
MNXR17003
MNXR17004
MNXR17006
MNXR17007
MNXR17013
MNXR17016
MNXR17021
MNXR17022
MNXR17023
MNXR17025
MNXR17026
MNXR17031
MNXR17035
MNXR1704
MNXR17040
MNX

MNXR21355
MNXR21357
MNXR2136
MNXR21362
MNXR21368
MNXR21374
MNXR21375
MNXR21376
MNXR2138
MNXR21385
MNXR21386
MNXR21387
MNXR21389
MNXR2139
MNXR21395
MNXR21397
MNXR21399
MNXR214
MNXR21401
MNXR21402
MNXR21403
MNXR2141
MNXR21417
MNXR21421
MNXR2143
MNXR21432
MNXR21433
MNXR2144
MNXR21440
MNXR21441
MNXR21442
MNXR21447
MNXR21449
MNXR21450
MNXR21453
MNXR21454
MNXR21455
MNXR21456
MNXR21457
MNXR21458
MNXR21459
MNXR21460
MNXR21461
MNXR21462
MNXR21463
MNXR21464
MNXR21465
MNXR21466
MNXR21467
MNXR21468
MNXR2147
MNXR21477
MNXR21478
MNXR2148
MNXR21486
MNXR21487
MNXR21488
MNXR21490
MNXR215
MNXR21502
MNXR21503
MNXR21506
MNXR21507
MNXR21508
MNXR2151
MNXR21510
MNXR21511
MNXR21512
MNXR21513
MNXR21517
MNXR2152
MNXR21524
MNXR21527
MNXR21536
MNXR21539
MNXR21547
MNXR21553
MNXR21555
MNXR21564
MNXR21566
MNXR21572
MNXR21582
MNXR21591
MNXR21592
MNXR21593
MNXR21594
MNXR21595
MNXR21596
MNXR21615
MNXR21616
MNXR21617
MNXR21618
MNXR21638
MNXR21657
MNXR21665
MNXR21666
MNXR21667
MNXR21668
MNXR21669
MNXR21671
MNXR21672
MNXR

MNXR24377
MNXR2438
MNXR24382
MNXR24388
MNXR24389
MNXR2439
MNXR24397
MNXR24398
MNXR24409
MNXR24411
MNXR24416
MNXR24417
MNXR24420
MNXR24421
MNXR24422
MNXR24423
MNXR24426
MNXR24427
MNXR24433
MNXR24434
MNXR24435
MNXR24436
MNXR2444
MNXR24445
MNXR24446
MNXR24447
MNXR24448
MNXR2445
MNXR24455
MNXR2446
MNXR24461
MNXR24463
MNXR2447
MNXR24473
MNXR24474
MNXR24476
MNXR2448
MNXR24484
MNXR24489
MNXR2449
MNXR24491
MNXR24492
MNXR24494
MNXR245
MNXR24502
MNXR24505
MNXR24513
MNXR24524
MNXR24526
MNXR24528
MNXR24532
MNXR24534
MNXR24536
MNXR24537
MNXR24538
MNXR2454
MNXR24544
MNXR24546
MNXR24548
MNXR2455
MNXR24554
MNXR24559
MNXR2456
MNXR24560
MNXR24562
MNXR24568
MNXR24570
MNXR24571
MNXR24572
MNXR24573
MNXR24574
MNXR24584
MNXR24591
MNXR24600
MNXR24604
MNXR24605
MNXR24624
MNXR24627
MNXR24629
MNXR24630
MNXR24631
MNXR24633
MNXR24635
MNXR24636
MNXR24644
MNXR24651
MNXR24657
MNXR24658
MNXR2466
MNXR24664
MNXR2467
MNXR24673
MNXR2468
MNXR24680
MNXR24681
MNXR24683
MNXR24684
MNXR24685
MNXR24686
MNXR24687
MNXR24688
MNXR24

MNXR27791
MNXR27793
MNXR27794
MNXR27795
MNXR278
MNXR2780
MNXR27800
MNXR27801
MNXR27802
MNXR27803
MNXR27804
MNXR27805
MNXR27807
MNXR27808
MNXR27809
MNXR2781
MNXR27810
MNXR27811
MNXR27812
MNXR27813
MNXR27814
MNXR27815
MNXR27816
MNXR27818
MNXR27819
MNXR2782
MNXR27820
MNXR27821
MNXR27822
MNXR27823
MNXR27824
MNXR27826
MNXR27827
MNXR27828
MNXR2783
MNXR27831
MNXR27832
MNXR27833
MNXR27835
MNXR27837
MNXR27841
MNXR27842
MNXR27843
MNXR27844
MNXR27845
MNXR27846
MNXR27849
MNXR27850
MNXR27851
MNXR27852
MNXR27853
MNXR27854
MNXR27856
MNXR27859
MNXR2786
MNXR27860
MNXR27862
MNXR27863
MNXR27864
MNXR27865
MNXR27868
MNXR2787
MNXR27876
MNXR2788
MNXR27883
MNXR27892
MNXR27893
MNXR27894
MNXR27895
MNXR27899
MNXR279
MNXR2790
MNXR27900
MNXR27904
MNXR27905
MNXR27906
MNXR27907
MNXR27922
MNXR27923
MNXR2793
MNXR27931
MNXR27932
MNXR2794
MNXR27947
MNXR2795
MNXR27955
MNXR27971
MNXR27973
MNXR27979
MNXR2798
MNXR27984
MNXR27986
MNXR2799
MNXR27996
MNXR27997
MNXR28
MNXR280
MNXR28001
MNXR28004
MNXR28005
MNXR28006
MNXR2801
MNX

...

MNXR90140
MNXR90141
MNXR90142
MNXR90143
MNXR90144
MNXR90145
MNXR90146
MNXR90147
MNXR90148
MNXR90149
MNXR90150
MNXR90151
MNXR90152
MNXR90153
MNXR90154
MNXR90155
MNXR90156
MNXR90157
MNXR90158
MNXR90159
MNXR9016
MNXR90160
MNXR90161
MNXR90162
MNXR90163
MNXR90164
MNXR90165
MNXR90166
MNXR90167
MNXR90168
MNXR90169
MNXR9017
MNXR90170
MNXR90171
MNXR90172
MNXR90173
MNXR90174
MNXR90175
MNXR90176
MNXR90177
MNXR90178
MNXR90179
MNXR9018
MNXR90180
MNXR90181
MNXR90182
MNXR90183
MNXR90184
MNXR90185
MNXR90186
MNXR90187
MNXR90188
MNXR90189
MNXR9019
MNXR90190
MNXR90191
MNXR90192
MNXR90193
MNXR90194
MNXR90195
MNXR90196
MNXR90197
MNXR90198
MNXR90199
MNXR902
MNXR9020
MNXR90200
MNXR90201
MNXR90202
MNXR90203
MNXR90204
MNXR90205
MNXR90206
MNXR90207
MNXR90208
MNXR90209
MNXR9021
MNXR90210
MNXR90211
MNXR90212
MNXR90213
MNXR90214
MNXR90215
MNXR90216
MNXR90217
MNXR90218
MNXR90219
MNXR90220
MNXR90221
MNXR90222
MNXR90223
MNXR90224
MNXR90225
MNXR90226
MNXR90227
MNXR90228
MNXR90229
MNXR9023
MNXR90230
MNXR90231
MNXR90232

MNXR91165
MNXR91166
MNXR91167
MNXR91168
MNXR91169
MNXR9117
MNXR91170
MNXR91171
MNXR91172
MNXR91173
MNXR91174
MNXR91175
MNXR91176
MNXR91177
MNXR91178
MNXR91179
MNXR9118
MNXR91180
MNXR91181
MNXR91182
MNXR91183
MNXR91184
MNXR91185
MNXR91186
MNXR91187
MNXR91188
MNXR91189
MNXR9119
MNXR91190
MNXR91191
MNXR91192
MNXR91193
MNXR91194
MNXR91195
MNXR91196
MNXR91197
MNXR91198
MNXR91199
MNXR91200
MNXR91201
MNXR91202
MNXR91203
MNXR91204
MNXR91205
MNXR91206
MNXR91207
MNXR91208
MNXR91209
MNXR91210
MNXR91211
MNXR91212
MNXR91213
MNXR91214
MNXR91215
MNXR91216
MNXR91217
MNXR91218
MNXR91219
MNXR91220
MNXR91221
MNXR91222
MNXR91223
MNXR91224
MNXR91225
MNXR91226
MNXR91227
MNXR91228
MNXR91229
MNXR91230
MNXR91231
MNXR91232
MNXR91233
MNXR91234
MNXR91235
MNXR91236
MNXR91237
MNXR91238
MNXR91239
MNXR91240
MNXR91241
MNXR91242
MNXR91243
MNXR91244
MNXR91245
MNXR91246
MNXR91247
MNXR91248
MNXR91249
MNXR91250
MNXR91251
MNXR91252
MNXR91253
MNXR91254
MNXR91255
MNXR91256
MNXR91257
MNXR91258
MNXR91259
MNXR91260
MNXR91261
MNX

MNXR92049
MNXR92050
MNXR92051
MNXR92052
MNXR92053
MNXR92054
MNXR92055
MNXR92056
MNXR92057
MNXR92058
MNXR92059
MNXR92060
MNXR92061
MNXR92062
MNXR92063
MNXR92064
MNXR92065
MNXR92066
MNXR92067
MNXR92068
MNXR92069
MNXR92070
MNXR92071
MNXR92072
MNXR92073
MNXR92074
MNXR92075
MNXR92076
MNXR92077
MNXR92078
MNXR92079
MNXR9208
MNXR92080
MNXR92081
MNXR92082
MNXR92083
MNXR92084
MNXR92085
MNXR92086
MNXR92087
MNXR92088
MNXR92089
MNXR9209
MNXR92090
MNXR92091
MNXR92092
MNXR92093
MNXR92094
MNXR92095
MNXR92096
MNXR92097
MNXR92098
MNXR92099
MNXR9210
MNXR92100
MNXR92101
MNXR92102
MNXR92103
MNXR92104
MNXR92105
MNXR92106
MNXR92107
MNXR92108
MNXR92109
MNXR9211
MNXR92110
MNXR92111
MNXR92112
MNXR92113
MNXR92114
MNXR92115
MNXR92116
MNXR92117
MNXR92118
MNXR92119
MNXR9212
MNXR92120
MNXR92121
MNXR92122
MNXR92123
MNXR92124
MNXR92125
MNXR92126
MNXR92127
MNXR92128
MNXR92129
MNXR9213
MNXR92130
MNXR92131
MNXR92132
MNXR92133
MNXR92134
MNXR92135
MNXR92136
MNXR92137
MNXR92138
MNXR92139
MNXR9214
MNXR92140
MNXR92141
MNXR921

MNXR92995
MNXR92996
MNXR92997
MNXR92998
MNXR92999
MNXR930
MNXR93000
MNXR93001
MNXR93002
MNXR93003
MNXR93004
MNXR93005
MNXR93006
MNXR93007
MNXR93008
MNXR93009
MNXR93010
MNXR93011
MNXR93012
MNXR93013
MNXR93014
MNXR93015
MNXR93016
MNXR93017
MNXR93018
MNXR93019
MNXR93020
MNXR93021
MNXR93022
MNXR93023
MNXR93024
MNXR93025
MNXR93026
MNXR93027
MNXR93028
MNXR93029
MNXR93030
MNXR93031
MNXR93032
MNXR93033
MNXR93034
MNXR93035
MNXR93036
MNXR93037
MNXR93038
MNXR93039
MNXR93040
MNXR93041
MNXR93042
MNXR93043
MNXR93044
MNXR93045
MNXR93046
MNXR93047
MNXR93048
MNXR93049
MNXR93050
MNXR93051
MNXR93052
MNXR93053
MNXR93054
MNXR93055
MNXR93056
MNXR93057
MNXR93058
MNXR93059
MNXR93060
MNXR93061
MNXR93062
MNXR93063
MNXR93064
MNXR93065
MNXR93066
MNXR93067
MNXR93068
MNXR93069
MNXR93070
MNXR93071
MNXR93072
MNXR93073
MNXR93074
MNXR93075
MNXR93076
MNXR93077
MNXR93078
MNXR93079
MNXR9308
MNXR93080
MNXR93081
MNXR93082
MNXR93083
MNXR93084
MNXR93085
MNXR93086
MNXR93087
MNXR93088
MNXR93089
MNXR9309
MNXR93090
MNXR93091
MNXR

MNXR93981
MNXR93982
MNXR93983
MNXR93984
MNXR93985
MNXR93986
MNXR93987
MNXR93988
MNXR93989
MNXR93990
MNXR93991
MNXR93992
MNXR93993
MNXR93994
MNXR93995
MNXR93996
MNXR93997
MNXR93998
MNXR93999
MNXR94
MNXR94000
MNXR94001
MNXR94002
MNXR94003
MNXR94004
MNXR94005
MNXR94006
MNXR94007
MNXR94008
MNXR94009
MNXR94010
MNXR94011
MNXR94012
MNXR94013
MNXR94014
MNXR94015
MNXR94016
MNXR94017
MNXR94018
MNXR94019
MNXR9402
MNXR94020
MNXR94021
MNXR94022
MNXR94023
MNXR94024
MNXR94025
MNXR94026
MNXR94027
MNXR94028
MNXR94029
MNXR9403
MNXR94030
MNXR94031
MNXR94032
MNXR94033
MNXR94034
MNXR94035
MNXR94036
MNXR94037
MNXR94038
MNXR94039
MNXR94040
MNXR94041
MNXR94042
MNXR94043
MNXR94044
MNXR94045
MNXR94046
MNXR94047
MNXR94048
MNXR94049
MNXR94050
MNXR94051
MNXR94052
MNXR94053
MNXR94054
MNXR94055
MNXR94056
MNXR94057
MNXR94058
MNXR94059
MNXR94060
MNXR94061
MNXR94062
MNXR94063
MNXR94064
MNXR94065
MNXR94066
MNXR94067
MNXR94068
MNXR94069
MNXR94070
MNXR94071
MNXR94072
MNXR94073
MNXR94074
MNXR94075
MNXR94076
MNXR94077
MNXR9

In [21]:
# PAL
rfp['MNXR7145'].ToBitString()

'000000000000000000000000000000000100000000000000000000000000000000000000000100000000000000000000100001000000000010000000001000000000011000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000010000000000000000000000001000000000000000000100001000100000000010000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000001000000100000000000000000000000000000000000010000000000000000100000000000000000000000000010000100000000000000000100000000000000000000000000000000000000000000000000000101000000000000000000000000000010000000010000000010000000000000010000000000000010100100000000000000000010000001000000000000000000010000001000000000000000000000000000000000000000000000000001000001110000000000000000000000000000000000000000000000000000000000000000000000001000000000000000100000001000010000000000000000000000000000000000000000000000000000000000000000000000

In [22]:
# TAL
rfp['MNXR8874'].ToBitString()

'000000000000000000000000000000000100000000000000000000000000000000000000000100000000000000000000100001000000000010000000001000000000011000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000010000000000000000000000001000000000000000000100001000100000000010000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000001000000000000000000000000000000000000000000010000000000000000100000000000000000000000000010000100000000000000000100000000000000000000000000000000000000000000000000000101000000000000000000000000000010000000010000000010000000000000010000000000000010100100000000000000000010000001000000000000000000010000001000000000000000000000000000000000000000000000000001000001110000000000000000000000000000000000000000000000000000000000000000000000001000000000000000100000001000010000000000000000000000000000000000000000000000000000000000000000000000

In [25]:
(rfp['MNXR7145'] ^ rfp['MNXR8874']).ToBitString()

'000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [31]:
(rfp['MNXR7145'] ^ rfp['MNXR8874']).GetNumOnBits()

2

In [32]:
palc = sorted(rfp, key=lambda x: (rfp['MNXR7145'] ^ rfp[x]).GetNumOnBits())

In [33]:
palc

['MNXR7145',
 'MNXR93681',
 'MNXR8874',
 'MNXR93685',
 'MNXR53231',
 'MNXR93744',
 'MNXR71242',
 'MNXR88847',
 'MNXR21351',
 'MNXR69930',
 'MNXR4257',
 'MNXR5171',
 'MNXR159',
 'MNXR20938',
 'MNXR76514',
 'MNXR23555',
 'MNXR3344',
 'MNXR272',
 'MNXR89883',
 'MNXR7124',
 'MNXR93683',
 'MNXR16093',
 'MNXR88228',
 'MNXR23495',
 'MNXR23395',
 'MNXR76515',
 'MNXR26946',
 'MNXR32826',
 'MNXR59176',
 'MNXR7534',
 'MNXR2964',
 'MNXR87207',
 'MNXR84191',
 'MNXR88849',
 'MNXR2963',
 'MNXR83116',
 'MNXR7538',
 'MNXR1347',
 'MNXR1349',
 'MNXR19851',
 'MNXR214',
 'MNXR2697',
 'MNXR30157',
 'MNXR30969',
 'MNXR4809',
 'MNXR4813',
 'MNXR4816',
 'MNXR58652',
 'MNXR60919',
 'MNXR60968',
 'MNXR616',
 'MNXR617',
 'MNXR619',
 'MNXR622',
 'MNXR623',
 'MNXR627',
 'MNXR633',
 'MNXR63429',
 'MNXR68217',
 'MNXR69455',
 'MNXR83292',
 'MNXR840',
 'MNXR845',
 'MNXR88230',
 'MNXR88846',
 'MNXR89688',
 'MNXR89746',
 'MNXR89747',
 'MNXR90472',
 'MNXR91348',
 'MNXR91350',
 'MNXR1356',
 'MNXR1364',
 'MNXR15211',
 'MNXR